In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

df = pd.read_csv('traffic.csv')

# Convert 'Time' to minutes
df['Time'] = pd.to_datetime(df['Time'], format='%I:%M:%S %p').dt.hour * 60 + pd.to_datetime(df['Time'], format='%I:%M:%S %p').dt.minute

# Preprocessing columns
numeric_features = ['Time', 'Date', 'CarCount', 'BikeCount', 'BusCount', 'TruckCount']
categorical_features = ['Day of the week']

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),  # Include 'Time' in numeric features
        ('cat', OneHotEncoder(), categorical_features)
    ])


X = df.drop("Total", axis='columns')  # Features
y = df["Total"]  # Target variable

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Create a pipeline with preprocessing and Linear Regression model
model = LinearRegression()

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Use the pipeline to make predictions on the testing set
predictions = pipeline.predict(X_test)

# Evaluate the model performance
mae = mean_absolute_error(y_test, predictions)


print(df.head())
# Display the evaluation metrics
print(f"Mean Absolute Error: {mae}") ##extermely small and means it is very close to actual values in testing set.


   Time  Date Day of the week  CarCount  BikeCount  BusCount  TruckCount  \
0     0    10         Tuesday        31          0         4           4   
1    15    10         Tuesday        49          0         3           3   
2    30    10         Tuesday        46          0         3           6   
3    45    10         Tuesday        51          0         2           5   
4    60    10         Tuesday        57          6        15          16   

   Total Traffic Situation  
0     39               low  
1     55               low  
2     55               low  
3     58               low  
4     94            normal  
Mean Absolute Error: 3.571810012124401e-14


In [6]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error

# Load the dataset
df = pd.read_csv('traffic.csv')

# Feature Engineering: Extract hour and minute from the 'Time' column
df['Hour'] = pd.to_datetime(df['Time'], format='%I:%M:%S %p').dt.hour
df['Minute'] = pd.to_datetime(df['Time'], format='%I:%M:%S %p').dt.minute

# Drop unnecessary columns
df.drop(['Time', 'Date'], axis='columns', inplace=True)

# Define feature columns
numeric_features = ['Hour', 'Minute', 'CarCount', 'BikeCount', 'BusCount', 'TruckCount']
categorical_features = ['Day of the week']

# Create a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

X = df.drop("Total", axis='columns')
y = df["Total"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Create a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Cross-validation
cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_absolute_error')
cv_mae = -cv_scores.mean()

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
pred = pipeline.predict(X_test)

# Evaluate the model performance using MAE
mae = mean_absolute_error(y_test, pred)

# Display evaluation metrics
print(f"Cross-validated MAE: {cv_mae}")
print(f"MAE on Test Set: {mae}")


Cross-validated Mean Absolute Error: 2.65804539507078
Mean Absolute Error on Test Set: 2.79026875699888
